# Import necessary python lib

In [23]:
import pandas as pd
import numpy as np

# Preprocessing

In [24]:
with open("x28.txt") as f:
    data = []
    content = f.readlines()[72:]
    for line in content:
        line  = line.strip()
        row = [float(i) for i in (line.split())[1:]]
        data.append(row)
    data = np.array(data)

# Get data

In [25]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,36.0,27.0,71.0,8.1,3.34,11.4,81.5,3243.0,8.8,42.6,11.7,21.0,15.0,59.0,59.0,921.870
1,35.0,23.0,72.0,11.1,3.14,11.0,78.8,4281.0,3.6,50.7,14.4,8.0,10.0,39.0,57.0,997.875
2,44.0,29.0,74.0,10.4,3.21,9.8,81.6,4260.0,0.8,39.4,12.4,6.0,6.0,33.0,54.0,962.354
3,47.0,45.0,79.0,6.5,3.41,11.1,77.5,3125.0,27.1,50.2,20.6,18.0,8.0,24.0,56.0,982.291
4,43.0,35.0,77.0,7.6,3.44,9.6,84.6,6441.0,24.4,43.7,14.3,43.0,38.0,206.0,55.0,1071.289


In [26]:
X = data[:, :-1]
Y = data[:, -1]
m = X.shape[0]
n = X.shape[1]

## Data description

In [27]:
print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape)
print("Number of samples m =", m)
print("Number of features n =", n)

Shape of X: (60, 15)
Shape of Y: (60,)
Number of samples m = 60
Number of features n = 15


# Normalize data

In [28]:
def normalize_and_add_ones(X):
    X_max = np.max(X, axis = 0)
    X_min = np.min(X, axis = 0)
    X_range = X_max - X_min
    X_normalized = (X - X_min) / (X_range)
    ones = np.ones((X_normalized.shape[0]))
    return np.column_stack((ones, X_normalized))

# Building model

In [32]:
class RidgeRegression:
    def __init__(self) -> None:
        pass
    def fit(self, X_train, Y_train, LAMBDA):
        assert len(X_train.shape) == 2 and X_train.shape[0] == Y_train.shape[0]
        W = np.linalg.pinv(X_train.T.dot(X_train) \
             + LAMBDA * np.identity(X_train.shape[1])).dot(X_train.T).dot(Y_train)
        # print(W)
        return W
    def predict(self, W, X_new):
        X_new = np.array(X_new)
        return W.dot(X_new.T)
    def compute_RSS(self, Y_new, Y_pred):
        m = Y_new.shape[0]
        loss = 1/m * (np.sum((Y_new - Y_pred)**2))
        return loss
    def get_the_best_LAMBDA(self, X_train, Y_train):

        def cross_validation(num_folds, LAMBDA):
            # get the index of all rows in the table
            row_ids = np.array(range(X_train.shape[0]))
            # valid end is the last index to get a valid split
            valid_end = len(row_ids) - len(row_ids) % num_folds
            valid_ids_set = np.split(row_ids[:valid_end], num_folds)
            # append the last index set which is not included in valid split above
            valid_ids_set.append(row_ids[valid_end:])
            # i-th training set may contain all number index k such that k not in i-th valid_ids_set
            train_ids_set = [[k for k in row_ids if k not in valid_ids_set[i]] for i in range (num_folds)]
            avg_RSS = 0
            for i in range (num_folds):
                valid_part = {"X": X_train[valid_ids_set[i]], "Y": Y_train[valid_ids_set[i]]}
                train_part = {"X": X_train[train_ids_set[i]], "Y": Y_train[train_ids_set[i]]}
                W = self.fit(train_part["X"], train_part["Y"], LAMBDA)
                Y_pred = self.predict(W, valid_part["X"])
                avg_RSS += self.compute_RSS(valid_part["Y"], Y_pred)
            return avg_RSS/(num_folds)
            
        def range_scan(best_LAMBDA, minimum_RSS, LAMBDA_values):
            for LAMBDA in LAMBDA_values:
                RSS = cross_validation(5, LAMBDA)
                if RSS < minimum_RSS:
                    best_LAMBDA = LAMBDA
                    minimum_RSS = RSS
            return best_LAMBDA, minimum_RSS
        best_LAMBDA, minimum_RSS = range_scan(best_LAMBDA = 0, minimum_RSS = np.inf, LAMBDA_values = np.array(range(50)))
        # after finding temp best_lambda, we continously scan from smaller range around best_lambda
        # (max(0, best_lambda - 1), best_lambda + 1) with step = 0.001
        LAMBDA_values = [k * 1./1000 for k in range(max(0, (best_LAMBDA - 1)*1000, \
                        (best_LAMBDA+1)*1000, 1))]
        best_LAMBDA, minimum_RSS = range_scan(best_LAMBDA, minimum_RSS, LAMBDA_values)
        
        return best_LAMBDA
            


In [30]:
X = normalize_and_add_ones(X)

In [35]:
X_train, Y_train = X[:50], Y[:50]
X_test, Y_test = X[50:], Y[50:]
ridge_regression = RidgeRegression()
best_LAMBDA = ridge_regression.get_the_best_LAMBDA(X_train, Y_train)
print("Best found LAMBDA", best_LAMBDA)
W = ridge_regression.fit(X_train, Y_train, LAMBDA = best_LAMBDA)
Y_pred = ridge_regression.predict(W, X_test)
RSS = ridge_regression.compute_RSS(Y_test, Y_pred)
print(RSS)
print(Y_pred)

Best found LAMBDA 0.002
1527.0698078029448
[877.39059783 921.33583769 931.7104582  893.81221771 993.93880571
 894.54583255 932.43571582 891.59941659 912.15753609 923.87639342]
